In [46]:
!pip install konlpy

In [47]:
import pandas as pd
import re
import konlpy
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
from tqdm import tqdm
import urllib.request

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-2, 2))

ratings_train.txt 파일을 탭(tab) 구분자로 읽어서 DataFrame으로 저장합니다.
전체 데이터 중 처음 10,000개 행(row)만 선택합니다.
원본 데이터에서 'id' 열과 'document' 열만 선택합니다.
나머지 열(예: 'label' 같은 것들)은 제거됩니다.
열 이름을 변경합니다.
여기서는 'document' 열의 이름을 'sentence'로 바꿔서 문장을 담는 열임을 명확하게 합니다.

In [55]:
# prompt: df의 데이터 에서  @아이디 부분을 전처리하여 제거하고 문장만 넣으세요 영어로 된 문장도 함께 삭제 합니다. 그리고  NaN값을 drop하세요

import pandas as pd
import re
import konlpy
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn.preprocessing import MinMaxScaler
!pip install konlpy

scaler = MinMaxScaler(feature_range=(-2, 2))
# ratings_train.txt 파일을 탭(tab) 구분자로 읽어서 DataFrame으로 저장합니다.
# 전체 데이터 중 처음 10,000개 행(row)만 선택합니다.
# 원본 데이터에서 'id' 열과 'document' 열만 선택합니다.
# 나머지 열(예: 'label' 같은 것들)은 제거됩니다.
# 열 이름을 변경합니다.
# 여기서는 'document' 열의 이름을 'sentence'로 바꿔서 문장을 담는 열임을 명확하게 합니다.
df = pd.read_excel('/content/유튜브_영상_댓글_수집_20250427140217.xlsx')
df = df["댓글 내용"].rename("sentence").to_frame()

# Regular expression to remove mentions (@username)
def remove_mentions(text):
    if isinstance(text, str):  # Check if the input is a string
        return re.sub(r'@\S+', '', text)
    else:
        return text

# Apply the function to the 'sentence' column
df['sentence'] = df['sentence'].apply(remove_mentions)

# Remove English sentences
def remove_english(text):
    if isinstance(text, str):
      if re.search('[a-zA-Z]', text):
        return ''
      else: return text
    else:
        return text

df['sentence'] = df['sentence'].apply(remove_english)

# Drop rows with NaN values in the 'sentence' column
df.dropna(subset=['sentence'], inplace=True)

#Remove empty strings
df = df[df['sentence'] != '']

df

,sentence
1,무안공항 참사는 결코 실수로 일어나지 않았습니다. 전국의 몇몇공항에도 콘크리트 둔덕...
2,굳이 이걸 또 기사화 하는 이유가 뭔지..사고이유도 다 밝혀진 마당에
3,사고 이유 조사 중인데요
4,버드스트라이크로 인한 엔진정지로 결론난거 아니였나요?\n게다가 저 장면이 사고 ...
5,국토부 항공철도사고조사 보고서 나왔나요? 아닌거로 아는데요
...,...
95,저 콘크리트 둔덕 넘어가면 어차피 일반 가옥이 기다리고 있음 더큰사고
96,비행기 + 공항 둘다입니다.\n비행기는 바퀴로 착륙해야해요
97,무죄추정의 원칙 모름?ㅋㅋㅋ 그짝들이 윤 탄핵전까지 써먹던 용어인데ㅋㅋㅋ 하물...
98,안전 장치들이 몇 겹으로 되어있는 이유는 그런 사고들이 있어왔기 때문입니다. ...


In [50]:
# prompt: df를 텍스트 파일 형태로 뽑아내세요
# 텍스트 파일 내용: 각 줄마다 감성 분석을 원하는 문장을 입력합니다. 예를 들어, 아래와 같이 감성 분석을 원하는 문장들을 입력한 text_data.txt 파일을 만들어주세요.
# 예시 (text_data.txt):
# 오늘 날씨가 너무 좋아서 기분이 좋다. 이 영화는 정말 별로였어. 너무 지루했다. 맛있는 음식을 먹어서 행복하다. 이 책은 이해하기 어려워서 실망했다.
# 각 문장은 줄 바꿈으로 구분되어야 합니다. 파일을 저장할 때, UTF-8 인코딩을 사용하여 저장해주세요.
# 다운로드하는 코드를 작성하시오

from google.colab import files

# Save the DataFrame to a text file with UTF-8 encoding
df.to_csv('text_data.txt', index=False, header=False, sep='\t', encoding='utf-8')

# Download the text file
files.download('text_data.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [56]:
sent_dic = pd.read_csv('/content/SentiWord_Dict.txt',sep = '\t',header=None)
sent_dic.iloc[0,0]='갈등'

pos_dic = sent_dic[sent_dic[1]>0]
neg_dic = sent_dic[sent_dic[1]<0]
neu_dic = sent_dic[sent_dic[1]==0]

In [57]:
class Aurora3:
    def __init__(self, df,sent_dic):
        self.df = df
        # 'sentence' 열이 존재하는지 확인하고, 없으면 인덱스라고 가정합니다.
        if isinstance(self.df, pd.Series):  # df가 시리즈인지 확인
            self.df = self.df.to_frame()  # 시리즈를 데이터프레임으로 변환
            self.df.index.name = 'index'
            self.df.columns = ['sentence']
        elif 'sentence' not in self.df.columns:
            self.df['sentence'] = self.df.index
        self.okt = konlpy.tag.Okt()
        self.sent_dic = sent_dic

    def get_df(self):# 최종 결과 반환
        print("문장을 토큰화 중입니다...")
        self.tokenizer_run()

        print("감성사전을 업데이트 중입니다...")
        self.expand_sent_dic()

        print("문장 감성분석 중입니다....")
        self.sent_analyze()
        return self.df

    def tokenizer_run(self):
        tqdm.pandas()

        def text_preprocess(x):
            # re.sub를 적용하기 전에 x가 문자열인지 확인
            if isinstance(x, str):
                text = []
                a = re.sub('[^가-힣0-9a-zA-Z\\s]', '', x)
                for j in a.split():
                    text.append(j)
                return ' '.join(text)
            else:
                # 문자열이 아닌 값 처리
                return ''


        def tokenize(x):
            text = []
            tokens = self.okt.pos(x)
            for token in tokens :
                if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
                    text.append(token[0])
            return text
        self.df['comment'] = self.df['sentence'].apply(lambda x : text_preprocess(x))
        self.df['comment'] = self.df['comment'].progress_apply(lambda x: tokenize(x))

    def expand_sent_dic(self):
        sent_dic = self.sent_dic

        def make_sent_dict(x) :
            pos=[]
            neg=[]
            tmp={}

            for sentence in tqdm(x):
                for word in sentence :
                    target = sent_dic[sent_dic[0]==word]
                    if len(target)==1: # 기존에 있는 단어라면 그대로 사용
                        score = float(target[1])
                        if score > 0:
                            pos.append(word)
                        elif score < 0:
                            neg.append(word)
                    tmp[word] = {'W':0,'WP':0,'WN':0} # 감성사전 구성
            pos = list(set(pos))
            neg = list(set(neg))

            for sentence in tqdm(x):
                for word in sentence :
                    tmp[word]['W'] += 1 # 빈도 수
                    for po in pos :
                        if po in sentence:
                            tmp[word]['WP'] += 1 # 긍정단어과 같은 문장 내 단어일 때
                            break
                    for ne in neg:
                        if ne in sentence:
                            tmp[word]['WN'] += 1 # 부정단어와 같은 문장내 단어일 때
                            break
            return pos, neg, pd.DataFrame(tmp)

        def make_score_dict(d,p,n):
            N=sum(d.iloc[0,::])
            pos_cnt=sum(d.loc[::,p].iloc[0,::])
            neg_cnt=sum(d.loc[::,n].iloc[0,::])

            trans =d.T
            trans['neg_cnt']=neg_cnt
            trans['pos_cnt']=pos_cnt
            trans['N']=N

            trans['MI_P']=np.log2(trans['WP']*trans['N']/trans['W']*trans['pos_cnt'])
            trans['MI_N']=np.log2(trans['WN']*trans['N']/trans['W']*trans['neg_cnt'])
            trans['SO_MI']=trans['MI_P'] - trans['MI_N']

            trans = trans.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
            trans = trans.sort_values(by=['SO_MI'],ascending=False)
            return trans

        def update_dict(d):
            add_Dic = {0:[],1:[]}
            for i in d.T.items():
                if i[0] not in list(sent_dic[0]):
                    if len(i[0]) > 1:
                        add_Dic[0].append(i[0])
                        add_Dic[1].append(i[1]['SO_MI'])

            add_Dic=pd.DataFrame(add_Dic)
            Sentiment=pd.merge(sent_dic,add_Dic,'outer')
            return Sentiment

        self.pos, self.neg, self.new_dict = make_sent_dict(self.df['comment'].values)

        self.t_dict = make_score_dict(self.new_dict,self.pos,self.neg)
        self.t_dict['SO_MI'] = scaler.fit_transform(self.t_dict['SO_MI'].values.reshape(-1,1))

        self.add_dict =update_dict(self.t_dict)

    def sent_analyze(self): # 데이터 감성분석
        tqdm.pandas()

        def get_cnt(x):
            cnt = 0
            for word in list(set(x)):
                target = self.add_dict[self.add_dict[0]==word]
                if len(target)==1:
                    cnt += float(target[1])
            return cnt

        def get_ratio(x):
            score = x['score']
            length = np.log10(len(x['comment']))+1
            try:
                ratio= round(score/length,2)
            except:
                ratio = 0
            return ratio

        tqdm.pandas()
        self.df['score']= self.df['comment'].progress_apply(lambda x : get_cnt(x))
        self.df['ratio'] = self.df.apply(lambda x: get_ratio(x), axis = 1)

In [58]:
test = Aurora3(df,sent_dic)
res = test.get_df()

문장을 토큰화 중입니다...


100%|██████████| 96/96 [00:00<00:00, 132.90it/s]


감성사전을 업데이트 중입니다...


  0%|          | 0/96 [00:00<?, ?it/s]<ipython-input-57-9ef542cd0c7d>:63: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  score = float(target[1])
100%|██████████| 96/96 [00:00<00:00, 3277.12it/s]
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-57-9ef542cd0c7d>:112: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  Sentiment=pd.merge(sent_dic,add_Dic,'outer')


문장 감성분석 중입니다....


  0%|          | 0/96 [00:00<?, ?it/s]<ipython-input-57-9ef542cd0c7d>:130: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  cnt += float(target[1])
100%|██████████| 96/96 [00:02<00:00, 34.79it/s]


In [59]:
res


,sentence,comment,score,ratio
1,무안공항 참사는 결코 실수로 일어나지 않았습니다. 전국의 몇몇공항에도 콘크리트 둔덕...,"[무안공항, 참사, 는, 결코, 실수, 로, 일어나지, 않았습니다, 전국, 의, 몇...",4.629055,1.63
2,굳이 이걸 또 기사화 하는 이유가 뭔지..사고이유도 다 밝혀진 마당에,"[굳이, 이, 걸, 또, 기사화, 하는, 이유, 가, 뭔, 지, 사, 고, 이유, ...",-0.652685,-0.29
3,사고 이유 조사 중인데요,"[사고, 이유, 조사, 중, 인데, 요]",-0.359198,-0.20
4,버드스트라이크로 인한 엔진정지로 결론난거 아니였나요?\n게다가 저 장면이 사고 ...,"[버드스트라이크, 로, 인한, 엔진, 정지, 로, 결론, 난, 거, 아니였나요, 게...",-0.300657,-0.13
5,국토부 항공철도사고조사 보고서 나왔나요? 아닌거로 아는데요,"[국토부, 항공, 철도사고, 조사, 보고서, 나왔나요, 아닌거로, 아는데요]",0.000000,0.00
...,...,...,...,...
95,저 콘크리트 둔덕 넘어가면 어차피 일반 가옥이 기다리고 있음 더큰사고,"[저, 콘크리트, 둔덕, 넘어가면, 어차피, 일반, 가옥, 이, 기다리고, 있음, ...",-0.486853,-0.23
96,비행기 + 공항 둘다입니다.\n비행기는 바퀴로 착륙해야해요,"[비행기, 공항, 둘다, 입니다, 비행기, 는, 바퀴, 로, 착륙, 해야, 해, 요]",-5.726510,-2.75
97,무죄추정의 원칙 모름?ㅋㅋㅋ 그짝들이 윤 탄핵전까지 써먹던 용어인데ㅋㅋㅋ 하물...,"[무죄, 추정, 의, 원칙, 모름, 그, 짝, 들, 이, 윤, 탄핵, 전, 까지, ...",-1.536913,-0.63
98,안전 장치들이 몇 겹으로 되어있는 이유는 그런 사고들이 있어왔기 때문입니다. ...,"[안전, 장치, 들, 이, 몇, 겹, 으로, 되어있는, 이유, 는, 그런, 사고, ...",10.399328,3.52
